In [1]:
from torch.nn.utils.rnn import pack_padded_sequence
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
from nltk.tokenize import word_tokenize
import h5py
from PIL import Image
from torch.autograd import Variable


from naive import Enc, Dec

import matplotlib.pyplot as plt

%matplotlib inline


In [4]:

from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import numpy as np

max_seq_len = 3
batch_size = 4
layer_num = 2

input_size = 10 # 0 - 9
emb_size = 8
hidden_size = 10

# test data:
a = [[1,2,3], [4,5,0], [6,0,0], [7,8,9]]
lens = [3,2,1,3]

# sort the input batch data by reversed actual length for pad_pack operation
pairs = sorted( zip(a, lens), key=lambda p: p[1], reverse=True)
(a, lens) = zip(*pairs)

# actual length
lens = np.array(lens)
# lens = torch.LongTensor(lens)

va = Variable(torch.LongTensor(a))
vlens = Variable(torch.LongTensor(lens))

print(va, vlens)

embedding = nn.Embedding(input_size, emb_size)
gru = nn.GRU(emb_size, hidden_size, layer_num)

inputs = va.transpose(0, 1); print("inputs size: ", inputs.size()) # max_seq_len * batch_size
inputs = embedding(inputs); print("embedded size: ", embedding.size()) # max_seq_len * batch_size * emb_size

inputs = torch.nn.utils.rnn.pack_padded_sequence(inputs, lens)

h0 = Variable(torch.randn(layer_num, 4, 10))
outputs, hn = gru(inputs, h0)

# print("after packed:")
# print("outputs.size: ", outputs.size())
# print("hn size: ", hn.size())

tmp = torch.nn.utils.rnn.pad_packed_sequence(outputs)

outputs, output_lengths = tmp
# outputs: (max_seq_len * batch_size, output_size)

print("after padded:")
print("outputs.size: ", outputs.size())
print("hn size: ", hn.size())

# outputs.index_select(torch.LongTensor(lens-1))
# print("lens: ", lens-1)

# idxs = Variable(torch.LongTensor(lens-1))
# outputs = outputs.index_select(0, idxs)

# print("selected outputs size: ", outputs.size())
# outputs.gather(0, idxs)

(tensor([[1, 2, 3],
        [7, 8, 9],
        [4, 5, 0],
        [6, 0, 0]]), tensor([3, 3, 2, 1]))
('inputs size: ', torch.Size([3, 4]))


AttributeError: 'Embedding' object has no attribute 'size'

In [ ]:
# TEST_IMG = "COCO_val2014_000000000042.jpg"
# pil_im = Image.open('data/val2014/COCO_val2014_000000000042.jpg', 'r')
# plt.figure()
# plt.imshow(np.asarray(pil_im))

# pil_im2= pil_im.resize((256, 256), Image.ANTIALIAS)
# im2a = np.array(pil_im2)
# # imshow(np.asarray(pil_im))
# print im2a.shape
# plt.figure()
# plt.imshow(im2a)
